# MNIST: MLP ensemble

## Required imports

In [2]:
from collections import Counter
from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import load_model, Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split

## Data preparation

For convenience, we define a function that does the data preparation for this learning task.

In [3]:
def prepare_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype(np.float32)/255.0
    x_test = x_test.astype(np.float32)/255.0
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return x_train, y_train, x_test, y_test

In [4]:
X_train, Y_train, x_test, y_test = prepare_data()

## Model definition

Since a model will have to be created for each ensemble member, we define a function that defines and compiles our model.

In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=SGD(),
                  metrics=['accuracy'])
    return model

## Training

For reproducibility, we seed the random number generator.

In [6]:
np.random.seed(45984)

We train three models, and store them in a list.  Note that each is trained on a different training set, since `train_test_split` splits input/output into random subsets.

In [7]:
nr_models = 3
models = list()
for i in range(nr_models):
    print(f'model {i + 1}')
    model = create_model()
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train)
    _ = model.fit(x_train, y_train, batch_size=64, epochs=10,
                  validation_data=(x_val, y_val), verbose=0)
    models.append(model)

model 1


2024-03-20 09:54:27.891996: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-20 09:54:28.321918: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-20 09:54:28.321969: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-20 09:54:28.326420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-20 09:54:28.326464: I external/local_xla/xla/stream_executor

model 2
model 3


## Evaluation

We compute and show the accuracies of the individual models on the test data.

In [8]:
accuracies = list()
for model in models:
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    accuracies.append(accuracy)
accuracies

[0.9503999948501587, 0.9484999775886536, 0.9495999813079834]

Next, we compute the prediction of each individual model on the test input.

In [10]:
y_infer = np.empty((len(models), len(y_test)), dtype=np.int64)
for i, model in enumerate(models):
    prediction = model.predict(x_test)
    y_infer[i, :] = np.argmax(prediction, axis=1)

313/313 [==============================] - 0s 1ms/step


We evaluate the ensemble by using the majority vote for the individuals.

In [11]:
mistakes = 0
for j in range(y_infer.shape[1]):
    counter = Counter()
    for i in range(y_infer.shape[0]):
        counter[y_infer[i, j]] += 1
    inferred = counter.most_common(1)[0][0]
    if inferred != np.argmax(y_test[j]):
        mistakes += 1
print(1.0 - mistakes/len(y_test))

0.951


The accuracy is indeed (marginally) better than that of the most accurate individual model.

## However...

For the same investment of compute time, you might expect that training a single model for three times as many epochs will yield a better result.

In [12]:
np.random.seed(45984)

In [13]:
model = create_model()
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train)
_ = model.fit(x_train, y_train, batch_size=64, epochs=30,
              validation_data=(x_val, y_val), verbose=0)

In [14]:
model.evaluate(x_test, y_test, verbose=0)

[0.09376676380634308, 0.9704999923706055]

That intuition would be entirely justified.